In [1]:
from init import *

In [2]:
def BuildModel_Simple():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [3]:
m = BuildModel_Simple()
m.evaluations.append(sdk.EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "BrownianMotion", "refs": [], "args": [0.2, 2], "start": 0.1}], "evaluations": [{"state": 0, "time": 2}]}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'has_state': [0, 1],
 'histograms': [],
 'mean': [None, 0.5377951860427856],
 'names': ['IndependentGaussian', 'BrownianMotion'],
 'npaths': [0, 10000],
 'skewness': [None, -0.044270601123571396],
 'stddev': [None, 2.8695006370544434],
 'time_points': [2.0],
 'time_steps': [9]}

In [5]:
import time
m = BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 1000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(sdk.EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'has_state': [0, 1],
 'histograms': [],
 'mean': [None,
  0.29666298627853394,
  None,
  0.49162912368774414,
  None,
  1.0941177606582642,
  None,
  2.093735456466675],
 'names': ['IndependentGaussian', 'BrownianMotion'],
 'npaths': [0, 1000000, 0, 1000000, 0, 1000000, 0, 1000000],
 'skewness': [None,
  -0.0016225617146119475,
  None,
  -0.0012908988865092397,
  None,
  0.002160679781809449,
  None,
  0.0010149761801585555],
 'stddev': [None,
  2.000231981277466,
  None,
  2.826359272003174,
  None,
  4.46881103515625,
  None,
  6.328188419342041],
 'time_points': [1.0, 2.0, 5.0, 10.0],
 'time_steps': [99, 199, 499, 999]}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

2.6 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
def TwoProcessesModel ():
    m = sdk.Model()
    m.TimeStart = 0
    m.TimeSteps = 10
    m.NumPaths = 10000
    m.Add(sdk.IndependentGaussian())
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    m.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
    return m

In [8]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 10
model.NumPaths = 10000
model.Add(sdk.IndependentGaussian())
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.Add(sdk.BrownianMotion(0.1,0.2,2)) # start, drift, diffusion
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(sdk.EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'has_state': [0, 1, 1],
 'histograms': [],
 'mean': [None,
  0.3199059069156647,
  0.32427075505256653,
  None,
  0.5528773069381714,
  0.5266329050064087,
  None,
  1.19133460521698,
  1.1671770811080933],
 'names': ['IndependentGaussian', 'BrownianMotion', 'BrownianMotion'],
 'npaths': [0, 10000, 10000, 0, 10000, 10000, 0, 10000, 10000],
 'skewness': [None,
  -0.004741800483316183,
  -0.014205324463546276,
  None,
  -0.0011879641097038984,
  -0.014298646710813046,
  None,
  0.012066695839166641,
  0.0350925549864769],
 'stddev': [None,
  2.0008509159088135,
  1.997839093208313,
  None,
  2.7948415279388428,
  2.828388214111328,
  None,
  4.426868915557861,
  4.430373668670654],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [1999, 3999, 9999]}

In [9]:
results = sdk.run (model, server)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,1,1.0,1999,10000,0.291903,0.020087,2.008664,-0.016391
1,BrownianMotion,,2,1.0,1999,10000,0.337432,0.019996,1.999585,0.009680
2,BrownianMotion,,1,2.0,3999,10000,0.473919,0.028026,2.802557,0.023502
3,BrownianMotion,,2,2.0,3999,10000,0.550611,0.028046,2.804574,0.033745
4,BrownianMotion,,1,5.0,9999,10000,1.079309,0.044273,4.427250,-0.017305
5,BrownianMotion,,2,5.0,9999,10000,1.148319,0.045165,4.516477,0.033050
